In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# 1. What are 3 areas on industry where computer vision is currently being used?

1. Streaming services
2. Social network services
3. Image generation

# 2. Search "what is overfitting in machine learning?" and write down a sentence about what you find.

Overfitting occurs when a model is excessively complex and has too many parameters relative to the size of the training data. As a result, the model may fit the training data very well, but it may no generalize well to new, unseen data. This means that the model will perform poorly on tasks it has not seen before, even though it may have a high accuracy on the training data.

# 3. Search "ways to prevent overfitting in machine learning" write down 3 of the things you find and a sentence about each.

1. Use cross-validation: Instead of splitting your data into a fixed training and testing set, you can use cross-validation to evaluate your model. This involves dividing your data into "folds", training on some folds, and evaluating on the remaining folds. You can then average the performance across all folds to get a better estimate of your model's generalization performance.
2. Ensemble methods: One way to reduce overfitting is to train multiple models and combine their predictions. This can be done by averaging the predictions of multiple models, or by training a higher-level model to make use of the predictions of multiple lower-level models.
3. Reducing the complexity of the model: A model with too many parameters may be prone to overefitting. One way to prevent this is to use a simpler model with fewer parameters, or to use techniques like feature selection to remove unnecessary features from the model.

# 4. Spend 20-minutes reading and clicking through the CNN Eplainer website

# 5. load the ```torchvision.datasets.MNIST``` train and test datasets.

In [ ]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

train_data = MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

# 6. Visualize at leat 5 different samples of the MNIST training dataset

In [ ]:
import matplotlib.pyplot as plt

row=1
column=5
for i in range(5):
    image, label = train_data[i]
    plt.subplot(row,column,i+1)
    plt.title(label)
    plt.axis(False)
    plt.imshow(image.squeeze(), cmap='gray')

# 7. Turn the MNIST train and test datasets into dataloaders using ```torch.utils.data.DataLoader```, set the ```batch_size=32```

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=32,
    shuffle=True,
)
test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=32,
)

# 8. Recreate model_2 used in this notebook capable of fitting on the MNIST dataset

In [ ]:
import torch.nn as nn

class TinyVGG(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()

        self.conv_stack_1 = nn.Sequential(
            nn.Conv2d(input_shape, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_stack_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units*7*7, output_shape)
        )

    def forward(self, x):
        x = self.conv_stack_1(x)
        x = self.conv_stack_2(x)
        x = self.classifier(x)

        return x

# 9. Train the model you built in exercise 8 on CPU and GPU and see how long it takes on each

In [ ]:
from helper_functions import train_step, test_step, print_train_time, eval_model
from timeit import default_timer as timer

In [ ]:
tinyvgg_cpu = TinyVGG(1, 10, len(train_data.classes)).to('cpu')
tinyvgg_gpu = TinyVGG(1, 10, len(train_data.classes)).to(device)

In [ ]:
from torchmetrics import Accuracy

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(tinyvgg_gpu.parameters(), lr=0.1)
acc_fn_gpu = Accuracy('multiclass', num_classes=len(train_data.classes)).to(device)
acc_fn_cpu = Accuracy('multiclass', num_classes=len(train_data.classes)).to('cpu')

In [ ]:
torch.manual_seed(42)

epochs = 3
train_start_gpu = timer()

for epoch in range(epochs):
    print(f'Epoch: {epoch}-----')
    train_step(tinyvgg_gpu, train_dataloader, loss_fn, optimizer, acc_fn_gpu, device)
    test_step(tinyvgg_gpu, test_dataloader, loss_fn, acc_fn_gpu, device)

train_end_gpu = timer()
total_train_time = print_train_time(train_start_gpu, train_end_gpu, device)

In [ ]:
torch.manual_seed(42)

epochs = 3
train_start_cpu = timer()

for epoch in range(epochs):
    print(f'Epoch: {epoch}-----')
    train_step(tinyvgg_cpu, train_dataloader, loss_fn, optimizer, acc_fn_cpu, 'cpu')
    test_step(tinyvgg_cpu, test_dataloader, loss_fn, acc_fn_cpu, 'cpu')

train_end_cpu = timer()
total_train_time = print_train_time(train_start_cpu, train_end_cpu, 'cpu')

# 10. Make predictions using your trained model and visualize at least 5 of them comparing the prediction to the target label

In [ ]:
tinyvgg_cpu_results = eval_model(tinyvgg_cpu, test_dataloader, loss_fn, acc_fn_cpu, 'cpu')
tinyvgg_gpu_results = eval_model(tinyvgg_gpu, test_dataloader, loss_fn, acc_fn_gpu, device)

In [ ]:
tinyvgg_cpu_results, tinyvgg_gpu_results

In [ ]:
torch.manual_seed(42)

X, y = next(iter(test_dataloader))
X, y = X.to(device), y.to(device)

pred_labels = None

tinyvgg_gpu.eval()
with torch.inference_mode():
    pred_logits = tinyvgg_gpu(X)
    pred_labels = pred_logits.argmax(dim=1)

X, y = X.cpu(), y.cpu()

for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(X[i].squeeze())
    plt.title(f'{pred_labels[i]} == {y[i]}')    
    plt.axis(False)

# 11. Plot confusion matrix comparing your model's predictions to the truth labels.

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

torch.manual_seed(42)

tinyvgg_gpu.eval()
preds_label = []
with torch.inference_mode():
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        preds_logits = tinyvgg_gpu(X)
        preds_label.append(preds_logits.argmax(dim=1).cpu())

pred_tensor = torch.concat(preds_label)

In [ ]:
confmat = ConfusionMatrix('multiclass', num_classes=len(test_data.classes))
confmat_tensor = confmat(preds=pred_tensor, target=test_data.targets)

fig, ax = plot_confusion_matrix(confmat_tensor.numpy(), class_names=test_data.classes)

# 12. Create a random tensor of shape [1,3,64,64] and pass it through a ```nn.Conv2d()``` layer with various hyperparameter settings (these can be any settings you choose), what do you notice if the ```kernel_size``` parameter goes up and down?

In [ ]:
random_tensor = torch.rand((1,3,64,64))

kernel_1 = nn.Conv2d(3, 10, 1, 1, 0)
kernel_3 = nn.Conv2d(3, 10, 3, 1, 0)
kernel_5 = nn.Conv2d(3, 10, 5, 1, 0)
kernel_7 = nn.Conv2d(3, 10, 7, 1, 0)

out1 = kernel_1(random_tensor)
out3 = kernel_3(random_tensor)
out5 = kernel_5(random_tensor)
out7 = kernel_7(random_tensor)

out1.shape, out3.shape, out5.shape, out7.shape

# 13. Use a model similar to the trained ```model_2``` from this notebook to make predictions on the test ```torchvision.datasets.FashionMNIST``` dataset.
- Then plot some predictions where the model was wrong alongside what the label of the image should've been.
- After visualizing these predictions do you think it's more of a modelling error or a data error?
- As in, could the model do better or are the labels of the data too close to each other.